In [1]:
import torch
from torch import nn
from torchvision.ops.misc import Permute
from torchvision.ops import StochasticDepth

In [2]:
class CNBlock(nn.Module):
    def __init__(self, in_channels, layer_scale, stochastic_depth_prob):
        super().__init__()

        self.residual = nn.Sequential(nn.Conv2d(in_channels, in_channels, 7, padding=3, groups=in_channels), # torchvision 구현체 보면 bias=True 임. 이유는? 잘 모르겠음
                                      Permute([0, 2, 3, 1]), # 개채행열 -> 개행열채 # torch.permute() 를 쓰려면 forward 에서 해야하기 때문에 nn.Sequential() 안에 넣으려면 이렇게!
                                      nn.LayerNorm(in_channels, eps=1e-6), # pixel-wise로 채널 축의 값들을 이용해서 normalize 하는 것
                                      # nn.LayerNorm 은 주어진 차원의 마지막부터 시작하여 역순으로 정규화를 수행한다.
                                      # 예를 들어, x = torch.randn(개, 채, 행, 열) 라면 layer_norm = nn.LayerNorm([채, 행, 열]) 이렇게 리스트로 줘야 함
                                      # (위는 채,행,열 에 대해서 전부다 평균 구하는 데 참여시키다는 뜻)
                                      # [개, 행, 열] 이런 식으로 건너뛸 수 없다는 것!
                                      Permute([0, 3, 1, 2]), # 개행열채 -> 개채행열
                                      nn.Conv2d(in_channels, 4 * in_channels, 1),
                                      nn.GELU(),
                                      nn.Conv2d(4 * in_channels, in_channels, 1))
        self.layer_scale = nn.Parameter(torch.ones(1,in_channels, 1, 1) * layer_scale)
        self.stochastic_depth = StochasticDepth(stochastic_depth_prob, "row")

    def forward(self, x):
        residual = self.layer_scale * self.residual(x) # 어떤 channel이 중요한지를 학습시키자 (SE Net 아이디어 비슷?)
        residual = self.stochastic_depth(residual)
        out = residual + x
        return out

class ConvNeXt(nn.Module):
    def __init__(self, block_setting, stochastic_depth_prob = 0.0, layer_scale = 1e-6, num_classes = 1000, **kwargs):
        super().__init__()

        layers = []
        layers += [nn.Sequential(nn.Conv2d(3, block_setting[0][0], kernel_size=4, stride=4),
                                 Permute([0, 2, 3, 1]),
                                 nn.LayerNorm(block_setting[0][0], eps=1e-6),
                                 Permute([0, 3, 1, 2]))]

        total_stage_blocks = sum([setting[2] for setting in block_setting])
        stage_block_id = 0
        for in_channels, out_channels, num_blocks in block_setting:
            stage = []
            for _ in range(num_blocks):
                sd_prob = stochastic_depth_prob * stage_block_id / (total_stage_blocks - 1) # 1 빼야 마지막 블록이 설정한 stochastic_depth_prob을 가지게 된다.
                stage.append(CNBlock(in_channels, layer_scale, sd_prob))
                stage_block_id += 1
            layers += [nn.Sequential(*stage)]
            if out_channels is not None:
                downsample = nn.Sequential(Permute([0, 2, 3, 1]),
                                           nn.LayerNorm(in_channels),
                                           Permute([0, 3, 1, 2]),
                                           nn.Conv2d(in_channels, out_channels, kernel_size=2, stride=2))
                layers += [downsample]

        self.features = nn.Sequential(*layers)

        # https://github.com/pytorch/vision/blob/main/torchvision/models/convnext.py#L160 참고
        # Swin과 달리 pre-activation이 아니라서 LN-GAP-fc 가 아님. 근데 GAP-fc 만 하기엔 마지막 LN이 멀어서 GAP-fc 사이에 LN을 추가한 듯
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(nn.LayerNorm(block_setting[-1][0]),
                                        nn.Linear(block_setting[-1][0], num_classes))

        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                nn.init.trunc_normal_(m.weight, std=0.02) # 논문엔 0.2 라 나와있는데 torchvision 코드는 0.02로 되어있음. 참고: https://github.com/pytorch/vision/blob/main/torchvision/models/swin_transformer.py#L603
                # timm 코드도 0.02 로 되어있어서 코드도 0.02로 반영. 참고: https://github.com/huggingface/pytorch-image-models/blob/4d9c3ae2fb7cc4739ec57d4c06254d2ffc7e2c89/timm/models/convnext.py#L380
                # head init scale 은 fine-tuning 때 하는 것이므로 여기선 생략. 적용 코드 참고: https://github.com/huggingface/pytorch-image-models/blob/4d9c3ae2fb7cc4739ec57d4c06254d2ffc7e2c89/timm/models/convnext.py#L383
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [3]:
def ConvNeXt_T(**kwargs):
    block_setting = [[96, 192, 3],
                     [192, 384, 3],
                     [384, 768, 9],
                     [768, None, 3]]
    return ConvNeXt(block_setting, stochastic_depth_prob = 0.1,  **kwargs)

def ConvNeXt_S(**kwargs):
    block_setting = [[96, 192, 3],
                     [192, 384, 3],
                     [384, 768, 27],
                     [768, None, 3]]
    return ConvNeXt(block_setting, stochastic_depth_prob = 0.4, **kwargs)

def ConvNeXt_B(**kwargs):
    block_setting = [[128, 256, 3],
                     [256, 512, 3],
                     [512, 1024, 27],
                     [1024, None, 3]]
    return ConvNeXt(block_setting, stochastic_depth_prob = 0.5, **kwargs)

def ConvNeXt_L(**kwargs):
    block_setting = [[192, 384, 3],
                     [384, 768, 3],
                     [768, 1536, 27],
                     [1536, None, 3]]
    return ConvNeXt(block_setting, stochastic_depth_prob = 0.5, **kwargs)

In [4]:
model = ConvNeXt_L()
# print(model)
!pip install torchinfo
from torchinfo import summary
summary(model, input_size=(2,3,224,224), device='cpu')

Layer (type:depth-idx)                        Output Shape              Param #
ConvNeXt                                      [2, 1000]                 --
├─Sequential: 1-1                             [2, 1536, 7, 7]           --
│    └─Sequential: 2-1                        [2, 192, 56, 56]          --
│    │    └─Conv2d: 3-1                       [2, 192, 56, 56]          9,408
│    │    └─Permute: 3-2                      [2, 56, 56, 192]          --
│    │    └─LayerNorm: 3-3                    [2, 56, 56, 192]          384
│    │    └─Permute: 3-4                      [2, 192, 56, 56]          --
│    └─Sequential: 2-2                        [2, 192, 56, 56]          --
│    │    └─CNBlock: 3-5                      [2, 192, 56, 56]          306,048
│    │    └─CNBlock: 3-6                      [2, 192, 56, 56]          306,048
│    │    └─CNBlock: 3-7                      [2, 192, 56, 56]          306,048
│    └─Sequential: 2-3                        [2, 384, 28, 28]          --
│

In [5]:
x = torch.randn(2,3,224,224)
print(model(x).shape)

torch.Size([2, 1000])


In [6]:
# Layer scale 기법에 대해
class MyModel(nn.Module):
    def __init__(self, in_channels, layer_scale):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.layer_scale = nn.Parameter(torch.ones(1, in_channels, 1, 1) * layer_scale)

    def forward(self, x):
        out = self.conv(x)
        print(out.shape)
        print(self.layer_scale.shape)
        print(self.layer_scale)
        print(out)
        out = out * self.layer_scale  # Apply layer scaling
        print(out)
        return out

model = MyModel(in_channels=2, layer_scale=0.5)
y=model(torch.randn(1, 2, 4, 4))
print(y.shape)

torch.Size([1, 2, 4, 4])
torch.Size([1, 2, 1, 1])
Parameter containing:
tensor([[[[0.5000]],

         [[0.5000]]]], requires_grad=True)
tensor([[[[ 0.1397,  0.0777, -0.0704, -0.1303],
          [ 0.0194, -0.4569, -0.4124, -0.5984],
          [ 0.5002,  0.5851,  0.5253,  0.5692],
          [ 0.2156,  0.6633, -0.5025, -0.0132]],

         [[ 0.0898,  0.6209, -0.1042,  0.6007],
          [ 0.2597,  0.0625, -1.1589, -0.1244],
          [-0.2417, -0.1112, -0.0112,  0.1503],
          [ 0.3058, -0.3266, -0.0510,  0.2003]]]],
       grad_fn=<ConvolutionBackward0>)
tensor([[[[ 0.0699,  0.0388, -0.0352, -0.0651],
          [ 0.0097, -0.2285, -0.2062, -0.2992],
          [ 0.2501,  0.2925,  0.2626,  0.2846],
          [ 0.1078,  0.3316, -0.2512, -0.0066]],

         [[ 0.0449,  0.3104, -0.0521,  0.3003],
          [ 0.1299,  0.0312, -0.5794, -0.0622],
          [-0.1208, -0.0556, -0.0056,  0.0751],
          [ 0.1529, -0.1633, -0.0255,  0.1002]]]], grad_fn=<MulBackward0>)
torch.Size([1, 2, 4, 4

In [7]:
y.sum().backward()
print(model.layer_scale.grad)

tensor([[[[1.1114]],

         [[0.1608]]]])


In [8]:
# Layer norm, 각 픽셀 위치 마다, 채널 축에 대해서 하려면?
x = torch.randn(2, 3, 2, 2)
ln = nn.LayerNorm(3, eps=1e-5)
y = ln(x.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
print("After LayerNorm: ", y) # 각 픽셀 위치에서 채널 축으로 평균 0 분산 1 이 되도록 함
print("weight shape: ", ln.weight.shape)

After LayerNorm:  tensor([[[[ 0.9828,  1.2877],
          [ 1.4139, -1.4134]],

         [[-1.3720, -0.1376],
          [-0.7306,  0.6678]],

         [[ 0.3892, -1.1501],
          [-0.6834,  0.7456]]],


        [[[ 1.2521,  0.4983],
          [-1.3906,  1.2001]],

         [[-0.0567, -1.3953],
          [ 0.4727,  0.0479]],

         [[-1.1954,  0.8970],
          [ 0.9179, -1.2480]]]], grad_fn=<PermuteBackward0>)
weight shape:  torch.Size([3])


In [9]:
# Layer norm, 각 픽셀 위치 마다, 채널 축에 대해서 하려면? (직접 구하기)
mean = x.mean(dim=1, keepdim=True)
std = x.std(dim=1, keepdim=True, unbiased=False)
print(mean.shape)
print(std.shape)
y_manual = (x - mean) / (std + 1e-5)
print("Manual normalization: ", y_manual)

torch.Size([2, 1, 2, 2])
torch.Size([2, 1, 2, 2])
Manual normalization:  tensor([[[[ 0.9829,  1.2877],
          [ 1.4139, -1.4135]],

         [[-1.3720, -0.1376],
          [-0.7306,  0.6678]],

         [[ 0.3892, -1.1501],
          [-0.6834,  0.7456]]],


        [[[ 1.2521,  0.4983],
          [-1.3906,  1.2001]],

         [[-0.0567, -1.3953],
          [ 0.4727,  0.0479]],

         [[-1.1954,  0.8970],
          [ 0.9179, -1.2480]]]])


In [10]:
# 배치놈은 어땟나
x = torch.randn(2, 3, 2, 2)
bn = nn.BatchNorm2d(3, eps=1e-5)
y_bn = bn(x)
print("After BatchNorm2d: ", y_bn)
print("weight shape: ", bn.weight.shape)

After BatchNorm2d:  tensor([[[[ 0.8974,  0.0792],
          [ 0.6630, -1.0971]],

         [[ 0.2158, -0.8629],
          [ 0.6147, -0.2307]],

         [[ 0.4788,  0.5985],
          [-1.6553,  1.2414]]],


        [[[-0.6904,  1.1693],
          [-1.7715,  0.7501]],

         [[ 0.0380,  1.5227],
          [-1.9934,  0.6959]],

         [[-1.2367,  0.1695],
          [ 1.0654, -0.6617]]]], grad_fn=<NativeBatchNormBackward0>)
weight shape:  torch.Size([3])


In [11]:
# 배치놈은 어땟나 (직접 구하기)
mean = x.mean(dim=(0, 2, 3), keepdim=True)
std = x.std(dim=(0, 2, 3), keepdim=True, unbiased=False)
print(mean.shape)
print(std.shape)
y_manual = (x - mean) / (std + 1e-5)
print("Manual Batch Normalization: ", y_manual)

torch.Size([1, 3, 1, 1])
torch.Size([1, 3, 1, 1])
Manual Batch Normalization:  tensor([[[[ 0.8974,  0.0792],
          [ 0.6629, -1.0971]],

         [[ 0.2158, -0.8629],
          [ 0.6147, -0.2307]],

         [[ 0.4788,  0.5985],
          [-1.6553,  1.2414]]],


        [[[-0.6904,  1.1693],
          [-1.7715,  0.7501]],

         [[ 0.0380,  1.5227],
          [-1.9934,  0.6959]],

         [[-1.2367,  0.1695],
          [ 1.0654, -0.6617]]]])


In [12]:
# 흔히 알려진 layer norm 그림에선..
x = torch.randn(2, 3, 2, 2)
ln = nn.LayerNorm([3, 2, 2], eps=1e-5, )
y = ln(x)
print("After LayerNorm: ", y)
print("weight shape: ", ln.weight.shape) # nn.LayerNorm([C,H,W]) 이면 C*H*W 개에 대해서 평균, 분산 내서 normalize 하는 거고 재배치할 평균, 분산도 각각 CHW 개다

After LayerNorm:  tensor([[[[-0.2695,  0.7893],
          [-0.0237, -0.3577]],

         [[-1.4171, -1.6256],
          [ 1.4453, -0.0731]],

         [[-0.1290, -0.8005],
          [ 1.8404,  0.6214]]],


        [[[ 1.2495, -1.5074],
          [ 0.7325,  1.2575]],

         [[ 0.8298,  0.8966],
          [-1.4235,  0.0346]],

         [[-0.0081, -1.4134],
          [-0.7245,  0.0763]]]], grad_fn=<NativeLayerNormBackward0>)
weight shape:  torch.Size([3, 2, 2])


In [13]:
# 흔히 알려진 layer norm 그림에선.. (직접 구하기)
mean = x.mean(dim=(1, 2, 3), keepdim=True)
std = x.std(dim=(1, 2, 3), keepdim=True, unbiased=False)
print(mean.shape)
print(std.shape)
y_manual = (x - mean) / (std + 1e-5)
print("Manual normalization: ", y_manual)

torch.Size([2, 1, 1, 1])
torch.Size([2, 1, 1, 1])
Manual normalization:  tensor([[[[-0.2695,  0.7893],
          [-0.0237, -0.3577]],

         [[-1.4171, -1.6256],
          [ 1.4453, -0.0731]],

         [[-0.1290, -0.8005],
          [ 1.8404,  0.6214]]],


        [[[ 1.2495, -1.5074],
          [ 0.7325,  1.2575]],

         [[ 0.8298,  0.8966],
          [-1.4234,  0.0346]],

         [[-0.0081, -1.4134],
          [-0.7245,  0.0763]]]])


In [14]:
# 내 필기용
from functools import partial

def power(base, exponent):
    return base ** exponent

square = partial(power, 2)  # 밑을 2로 고정
cube = partial(power, 3)  # 밑을 3으로 고정

print(square(4))  # 출력: 16 (2의 4제곱)
print(cube(3))  # 출력: 27 (3의 3제곱)

16
27
